In [66]:
import ollama
import yfinance as yf
import pandas as pd
from tqdm import tqdm
from textblob import TextBlob
import json

In [67]:
CRYPTO_SYMBOLS = {"Bitcoin": "BTC-USD", "Ethereum": "ETH-USD", "Ripple": "XRP-USD"}
SENTIMENTS = ["positive", "neutral", "negative"]
SENTIMENT_COLORS = {"positive": "green", "neutral": "gray", "negative": "red"}

In [68]:
def retrieve_news(selected_crypto, model):
    crypto_news = yf.Ticker(CRYPTO_SYMBOLS[selected_crypto]).get_news(1000)

    df = pd.DataFrame(
        [
            {
                "date": item["content"].get("pubDate", ""),
                "title": item["content"].get("title", ""),
                "text": item["content"].get("summary", ""),
                "url": item["clickThroughUrl"]["url"] if item.get("clickThroughUrl") else "",
            }
            for item in crypto_news
        ]
    )

    df["date"] = pd.to_datetime(df["date"])

    X_pred = df.drop(columns=["date"])

    return df

df = retrieve_news("Bitcoin", "")
df.head()

,date,title,text,url
0,2025-01-03 17:00:00+00:00,MicroStrategy history: Beyond the Ticker,"MicroStrategy (MSTR) and its chairman, Michael...",
1,2025-01-03 16:39:04+00:00,BlackRock's bitcoin ETF sees record daily outf...,Traders pulled $333 million from BlackRock's s...,
2,2025-01-03 14:54:00+00:00,XRP Price Rises. What’s Driving the Crypto’s R...,"The token is up about 15% already this year, m...",
3,2025-01-02 17:14:39+00:00,"Bitcoin, Vistra, Topgolf Callaway: Market Minute",Yahoo Finance host Madison Mills examines seve...,
4,2025-01-02 14:48:43+00:00,"Dow, stocks open higher as bitcoin bounces abo...",The Dow Jones Industrial Average is leading ga...,


In [69]:
MODEL = "llama3.1:latest"
SYSTEM_INSTRUCTIONS = "You are a helpful assistant specialized in analyzing news articles. Provide concise and accurate responses in JSON format."

ollama.pull(MODEL)

ProgressResponse(status='success', completed=None, total=None, digest=None)

In [70]:
def analyze_sentiment_textblob(text):
    blob = TextBlob(text)
    sentiment_score = blob.sentiment.polarity
    if sentiment_score > 0.1:
        return "positive", sentiment_score
    elif sentiment_score < -0.1:
        return "negative", sentiment_score
    else:
        return "neutral", sentiment_score

def analyze_sentiment(text):
    try:
        prompt = (
            f"{SYSTEM_INSTRUCTIONS}\n\n"
            f"Analyze the following blog article and return a JSON object with the following fields:\n"
            f"main_core_thoughts (a summary of the main core thoughts),\n"
            f"sentiment_analysis (sentiment as positive, neutral, or negative),\n"
            f"confidence_level (confidence score from 0 to 1),\n"
            f"topics (a list of up to 3 main topics discussed in the blog article).\n\n"
            f"Blog Article: {text}\n"
        )
        response = ollama.generate(model=MODEL, prompt=prompt, stream=False, format="json")
        response = json.loads(response.model_dump_json())
        return response
    except Exception as e:
        error = f"Error: {str(e)}"
        return error

In [71]:
for index, row in tqdm(df.iterrows(), desc="Analyzing sentiment"):
    response = analyze_sentiment(row["text"])
    response = json.loads(response["response"])
    for key, value in response.items():
        if key not in df.columns:
            df[key] = None
        df.at[index, key] = value

    break

df.head()

Analyzing sentiment: 0it [00:08, ?it/s]


,date,title,text,url,model,created_at,done,done_reason,total_duration,load_duration,prompt_eval_count,prompt_eval_duration,eval_count,eval_duration,response,context
0,2025-01-03 17:00:00+00:00,MicroStrategy history: Beyond the Ticker,"MicroStrategy (MSTR) and its chairman, Michael...",,llama3.1:latest,2025-01-03T22:15:34.4791942Z,True,stop,8366728100,41491300,629,258000000,143,8065000000,"{\n ""main_core_thoughts"": [\n ""MicroStrate...","[128006, 882, 128007, 271, 2675, 527, 264, 111..."
1,2025-01-03 16:39:04+00:00,BlackRock's bitcoin ETF sees record daily outf...,Traders pulled $333 million from BlackRock's s...,,None,None,None,None,None,None,None,None,None,None,None,None
2,2025-01-03 14:54:00+00:00,XRP Price Rises. What’s Driving the Crypto’s R...,"The token is up about 15% already this year, m...",,None,None,None,None,None,None,None,None,None,None,None,None
3,2025-01-02 17:14:39+00:00,"Bitcoin, Vistra, Topgolf Callaway: Market Minute",Yahoo Finance host Madison Mills examines seve...,,None,None,None,None,None,None,None,None,None,None,None,None
4,2025-01-02 14:48:43+00:00,"Dow, stocks open higher as bitcoin bounces abo...",The Dow Jones Industrial Average is leading ga...,,None,None,None,None,None,None,None,None,None,None,None,None
